## Importing the csv file

In [ ]:
#importing the file
from google.colab import files
uploaded = files.upload()



Saving reddit_suicide_detection_final_clean.csv to reddit_suicide_detection_final_clean.csv


In [ ]:
#installing the required transformers, datasets, and wandb libraries from hugging face
!pip -qqq install transformers datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
#import required packages and libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import ElectraTokenizerFast, ElectraForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
device = torch.device('cuda')

In [ ]:
#Loading the dataset
df = pd.read_csv('reddit_suicide_detection_final_clean.csv', header = 0)
#using pretrained model, we get rid of the embedding we created earlier
df = df.drop('cleaned_text', axis=1)
#convert the classes to 0: non suicide and 1: suicide
class_mapping = {'suicide': 1, 'non-suicide': 0}
# Replace to map class labels based on the dictionary
df['class'] = df['class'].replace(class_mapping)

df.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over... So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,It ends tonight.I can’t do it anymore. \nI quit.,1


In [ ]:
#Loading the tokenizer and the model
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-base-discriminator")
model = ElectraForSequenceClassification.from_pretrained("google/electra-base-discriminator", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Splitting the dataset
from sklearn.model_selection import train_test_split

# Splitting df into features and labels
X = df.drop('class', axis=1)  # This is your features matrix
y = df['class']  # These are your labels

# Splitting the data into training, validation, and testing sets
# First, split into training and a temporary set (combining validation and test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Then split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


# Extract the text column from the DataFrame as a list of strings
X_train= X_train['text'].tolist()
X_val= X_val['text'].tolist()
X_test = X_test['text'].tolist()

y_train = y_train.tolist()
y_val = y_val.tolist()
y_test = y_test.tolist()


In [ ]:
#tokenizing all of the data
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)


In [ ]:
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
#custom metrics for the trainer
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer and training arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
#train the model
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.237000
1000,0.218400
1500,0.160800
2000,0.161800
2500,0.157600
3000,0.156900
3500,0.144000
4000,0.153400
4500,0.133200
5000,0.128900


Step,Training Loss
500,0.237000
1000,0.218400
1500,0.160800
2000,0.161800
2500,0.157600
3000,0.156900
3500,0.144000
4000,0.153400
4500,0.133200
5000,0.128900


TrainOutput(global_step=17418, training_loss=0.12505710810741144, metrics={'train_runtime': 4430.0338, 'train_samples_per_second': 31.453, 'train_steps_per_second': 3.932, 'total_flos': 3.66618944538624e+16, 'train_loss': 0.12505710810741144, 'epoch': 1.0})

In [ ]:
#calculate the validation accuracy and scores
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.08696022629737854,
 'eval_accuracy': 0.9789860481139117,
 'eval_precision': 0.9719654041157173,
 'eval_recall': 0.9734169653524493,
 'eval_f1': 0.9726906431875839,
 'eval_runtime': 171.405,
 'eval_samples_per_second': 101.613,
 'eval_steps_per_second': 12.707,
 'epoch': 1.0}

In [ ]:
#save the model
trainer.save_model('final_electra_model')
tokenizer.save_pretrained('final_tokenizer')

('final_tokenizer/tokenizer_config.json',
 'final_tokenizer/special_tokens_map.json',
 'final_tokenizer/vocab.txt',
 'final_tokenizer/added_tokens.json',
 'final_tokenizer/tokenizer.json')

In [ ]:
#work on the test data
test_dataset = Dataset(X_test_tokenized, y_test)

In [ ]:
#demo inference to see if the model is accurate
text = "i am happy"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.0171, -3.2495]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9981, 0.0019]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.99810493, 0.00189505]], dtype=float32)

In [ ]:
import json
config  = json.load(open('Custom_electra_model/config.json'))


## Testing Accuracy

In [ ]:
saved_model = AutoModelForSequenceClassification.from_pretrained('Custom_electra_model')

In [ ]:
saved_trainer = Trainer(
    model=saved_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
saved_trainer.predict(test_dataset).metrics

<class 'transformers.trainer_utils.EvalPrediction'>


{'test_loss': 0.0993182510137558,
 'test_accuracy': 0.9761740727982546,
 'test_precision': 0.9704913121629718,
 'test_recall': 0.9674481110945199,
 'test_f1': 0.968967322216406,
 'test_runtime': 177.3651,
 'test_samples_per_second': 98.204,
 'test_steps_per_second': 12.28}

In [ ]:
!zip -r final_electra_model.zip /content/final_electra_model
!zip -r final_tokenizer.zip /content/final_tokenizer


  adding: content/final_electra_model/ (stored 0%)
  adding: content/final_electra_model/config.json (deflated 53%)
  adding: content/final_electra_model/model.safetensors (deflated 7%)
  adding: content/final_electra_model/training_args.bin (deflated 51%)
  adding: content/final_tokenizer/ (stored 0%)
  adding: content/final_tokenizer/vocab.txt (deflated 53%)
  adding: content/final_tokenizer/tokenizer_config.json (deflated 76%)
  adding: content/final_tokenizer/special_tokens_map.json (deflated 42%)
  adding: content/final_tokenizer/tokenizer.json (deflated 71%)


In [ ]:
from google.colab import files
files.download('final_electra_model.zip')
files.download('final_tokenizer.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>